In [1]:
# change the path to main folder

import os
os.chdir("D:\Cat_id\catid_pipelines\cloud_pipeline")

In [2]:
import argparse
import pathlib
from src.yolo.helpers import inference
from src.core.utils import get_package_path
from src.core.base_utils import silence_all_warnings
from production.conf.yolo_conf import *


In [3]:
import pandas as pd
from src.core.context import create_context as _create_context
from src.pipeline.main import (enroll_pet as _enroll_pet,
                                     infer_pet_id as _infer_pet_id,
                                    #  train_efficientnet as _train_efficientnet,
                                    )
from src.data_prep import db_utils
from src.data_prep import core
from src.pipeline import  efficientnet
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [4]:
if enroll:
    inference(weights, input_dir, output_dir, enroll, augment)

In [6]:
def run_enrollment(conf_path, re_enroll=False,augment=True):
    """Run the enrollment process on given raw pet images data for different households.

    Parameters
    ----------
    conf_path : str
       path of the project's `conf` folder
    re_enroll : bool
      whether to re-enroll the existing enrolled pets or not
      Note: To overwrite existing enrollment, set it to True

    """

    # create the project configuration object
    context_obj = _create_context(conf_path)

    # get list of existing enrolled pets
    enrolled_pets = []
    append_new_pets = False
    if not re_enroll:
        enrolled_pets = db_utils.load_enrolled_pet_db(context_obj)['house_pet_id'].unique()
        append_new_pets = True

    # prepare enroll metadata from the raw image data
    enroll_meta_df = core.prep_enroll_data(context_obj, enrolled_pets,augment)
    if len(enroll_meta_df) == 0:
        raise Warning(f"All the pets from the given enroll data are already enrolled! If you want to re-enroll, "
                      f"run the process again with re_enroll=True.")
    # save enroll metadata to the DB
    db_utils.save_enroll_metadata_db(context_obj, enroll_meta_df, append=append_new_pets)

    # load the trained pipeline models
    eff_net_model = efficientnet.load_model(context_obj)
    # yolo_model_objs = yolo.load_model(context_obj)

    # execute enroll process for each house
    house_dfs = []
    for house_id in enroll_meta_df['house_id'].unique():
        house_df = enroll_meta_df[enroll_meta_df['house_id'] == house_id]
        house_pet_db = _enroll_pet(context_obj, house_id, house_df, eff_net_model) #, yolo_model_objs)
        house_dfs.append(house_pet_db)

    # consolidate households pets into one and save the enrolled pet info to DB
    enrolled_pet_df = pd.concat(house_dfs, ignore_index=True)
    db_utils.save_enrolled_pet_db(context_obj, enrolled_pet_df, append=append_new_pets)
    print("Pet enrollment completed for all houses!")


def run_inference(conf_path, return_eval_report=False):
    """Run the inference process on given raw pet images data for different households.

    Parameters
    ----------
    conf_path : str
       path of the project's `conf` folder
    return_eval_report : bool
      whether to get classification evaluation report or not
      Note: Subject to if truth label instances are provided or not

    Returns
    -------
    report: dict or None
        a report dict if truth labels are provided otherwise None

    """

    # create the project configuration object
    context_obj = _create_context(conf_path)

    # prepare inference metadata from the raw image data
    infer_meta_df = core.prep_inference_data(context_obj)
    # save inference metadata to the DB
    db_utils.save_inference_metadata_db(context_obj, infer_meta_df)

    # load the trained pipeline models
    eff_net_model = efficientnet.load_model(context_obj)
    # yolo_model_objs = yolo.load_model(context_obj)

    # load enrolled pets db
    enrolled_pets_db = db_utils.load_enrolled_pet_db(context_obj)
    enrolled_houses = enrolled_pets_db['house_id'].unique()

    # execute inference process for each house
    house_dfs = []
    for house_id in infer_meta_df['house_id'].unique():
        if house_id not in enrolled_houses:
            # as no pets are enrolled from this house, inference can't be given for the same
            continue
        house_df = infer_meta_df[infer_meta_df['house_id'] == house_id]
        house_df_preds = _infer_pet_id(context_obj, house_id, house_df, enrolled_pets_db, eff_net_model)
                                    #    yolo_model_objs)
        house_dfs.append(house_df_preds)

    # consolidate households pets into one and save the inference data predictions to DB
    inference_preds_df = pd.concat(house_dfs, ignore_index=True)
    db_utils.save_inference_prediction_db(context_obj, inference_preds_df)

In [7]:
config_path = os.path.join(get_package_path(), "../production/conf/config.yml")

In [8]:
run_enrollment(config_path,re_enroll=True)

100%|██████████| 3/3 [00:00<00:00,  4.57it/s]


1/1 [==============================] - 0s 181ms/step
Pet enrollment completed for all houses!


In [9]:
run_inference(config_path)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.37it/s]


1 dog images detected and ignoring for inference.
1/1 [==============================] - 0s 161ms/step
(100, 64)
16 16
